In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Allergies"
cohort = "GSE182740"

# Input paths
in_trait_dir = "../../input/GEO/Allergies"
in_cohort_dir = "../../input/GEO/Allergies/GSE182740"

# Output paths
out_data_file = "../../output/preprocess/Allergies/GSE182740.csv"
out_gene_data_file = "../../output/preprocess/Allergies/gene_data/GSE182740.csv"
out_clinical_data_file = "../../output/preprocess/Allergies/clinical_data/GSE182740.csv"
json_path = "../../output/preprocess/Allergies/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll fix the issues with the file reading approach and simplify the clinical data creation using the sample characteristics dictionary directly.

```python
import pandas as pd
import numpy as np
import os
import gzip

# 1. Gene Expression Data Availability
# Based on the background information, this is microarray data for gene expression
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Allergies), we use the 'disease' field since this study compares different skin conditions
trait_row = 1  # The key for disease information

# No explicit age information is provided in the sample characteristics
age_row = None

# No gender information is provided in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert disease information to a binary indicator for allergies"""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Based on the study design, this dataset focuses on atopic dermatitis (a form of allergy)
    # and its overlap with psoriasis
    if value.lower() == 'atopic_dermatitis':
        return 1  # Allergic condition
    elif value.lower() == 'mixed':
        return 1  # This represents overlap phenotype which includes allergic component
    elif value.lower() == 'psoriasis':
        return 0  # Non-allergic skin condition
    elif value.lower() == 'normal_skin':
        return 0  # No allergic condition
    return None

def convert_age(value):
    """Convert age information to numeric value"""
    # Not applicable as age data is not available
    return None

def convert_gender(value):
    """Convert gender information to binary (0: female, 1: male)"""
    # Not applicable as gender data is not available
    return None

# 3. Save Metadata
# Conduct initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary directly
    sample_char_dict = {
        0: ['tissue: skin'], 
        1: ['disease: Psoriasis', 'disease: Atopic_dermatitis', 'disease: Mixed', 'disease: Normal_skin'],
        2: ['lesional (ls) vs. nonlesional (nl) vs. normal: LS', 'lesional (ls) vs. nonlesional (nl) vs. normal: NL', 'lesional (ls) vs. nonlesional (nl) vs. normal: Normal'],
        3: ['psoriasis area and diseave severity index (pasi): 10.1', 'psoriasis area and diseave severity index (pasi): 7.9', 'psoriasis area and diseave severity index (pasi): 10.4', 'psoriasis area and diseave severity index (pasi): 9', 'psoriasis area and diseave severity index (pasi): 18.4', 'psoriasis area and diseave severity index (pasi): 11.1', 'psoriasis area and diseave severity index (pasi): 8.5', 'psoriasis area and diseave severity index (pasi): 7.1', 'psoriasis area and diseave severity index (pasi): 6.3', 'psoriasis area and diseave severity index (pasi): 10.8', 'psoriasis area and diseave severity index (pasi): 7.4', 'psoriasis area and diseave severity index (pasi): 3.5', 'psoriasis area and diseave severity index (pasi): 4.7', 'psoriasis area and diseave severity index (pasi): 4', 'psoriasis area and diseave severity index (pasi): 25.4', 'psoriasis area and diseave severity index (pasi): 5.8', 'psoriasis area and diseave severity index (pasi): 6', 'psoriasis area and diseave severity index (pasi): 17.2', 'psoriasis area and diseave severity index (pasi): 7.6', 'psoriasis area and diseave severity index (pasi): 3.6', 'psoriasis area and diseave severity index (pasi): 2.4', 'psoriasis area and diseave severity index (pasi): 2.9', 'psoriasis area and diseave severity index (pasi): 17.9', 'psoriasis area and diseave severity index (pasi): 1.4', 'psoriasis area and diseave severity index (pasi): 18', 'psoriasis area and diseave severity index (pasi): 10.6', 'psoriasis area and diseave severity index (pasi): 11.8', 'psoriasis area and diseave severity index (pasi): 6.6', 'psoriasis area and diseave severity index (pasi): 20.4', 'psoriasis area and diseave severity index (pasi): 17.7'],
        4: ['scoring atopic dermatitis (scorad): 19.97', 'scoring atopic dermatitis (scorad): 41.94', 'scoring atopic dermatitis (scorad): 46.98', 'scoring atopic dermatitis (scorad): 36.38', 'scoring atopic dermatitis (scorad): 81.92', 'scoring atopic dermatitis (scorad): 39.24', 'scoring atopic dermatitis (scorad): 51.74', 'scoring atopic dermatitis (scorad): 17.03', 'scoring atopic dermatitis (scorad): 35.2', 'scoring atopic dermatitis (scorad): 29.64', 'scoring atopic dermatitis (scorad): 43.3', 'scoring atopic dermatitis (scorad): 42.97', 'scoring atopic dermatitis (scorad): 13.22', 'scoring atopic dermatitis (scorad): 13.87', 'scoring atopic dermatitis (scorad): 14.29', 'scoring atopic dermatitis (scorad): 36.44', 'scoring atopic dermatitis (scorad): 21.94', 'scoring atopic dermatitis (scorad): 18.62', 'scoring atopic dermatitis (scorad): 30.2', 'scoring atopic dermatitis (scorad): 17.14', 'scoring atopic dermatitis (scorad): 16.99', 'scoring atopic dermatitis (scorad): 14.51', 'scoring atopic dermatitis (scorad): 12.64', 'scoring atopic dermatitis (scorad): 16.33', 'scoring atopic dermatitis (scorad): 32.31', 'scoring atopic dermatitis (scorad): 14.52', 'scoring atopic dermatitis (scorad): 30.49', 'scoring atopic dermatitis (scorad): 29.03', 'scoring atopic dermatitis (scorad): 33.96', 'scoring atopic dermatitis (scorad): 12.76'],
        5: ['eczema area and severity index (easi): 9.4', 'eczema area and severity index (easi): 22.65', 'eczema area and severity index (easi): 25.55', 'eczema area and severity index (easi): 25.5', 'eczema area and severity index (easi): 47.65', 'eczema area and severity index (easi): 18.9', 'eczema area and severity index (easi): 28.65', 'eczema area and severity index (easi): 9.6', 'eczema area and severity index (easi): 20.95', 'eczema area and severity index (easi): 23.5', 'eczema area and severity index (easi): 29.6', 'eczema area an


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Load the input files to explore the dataset
import os
import pandas as pd
import re
import numpy as np

# Define possible paths for clinical data
clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
sample_info_path = os.path.join(in_cohort_dir, "sample_info.csv")
gene_data_path = os.path.join(in_cohort_dir, "gene_data.csv")

# Check if gene expression data is available
is_gene_available = os.path.exists(gene_data_path)

# Initialize variables
clinical_data = None
background_info = ""
sample_chars = {}

# Try to load clinical data from different possible files
if os.path.exists(clinical_data_path):
    clinical_data = pd.read_csv(clinical_data_path)
    print("Clinical data loaded from clinical_data.csv")
    # Display sample characteristics to identify relevant rows
    print("Sample characteristics preview:")
    for i, row in clinical_data.iterrows():
        unique_values = clinical_data.iloc[i, 1:].unique()
        sample_chars[i] = list(unique_values)
    print(sample_chars)
elif os.path.exists(sample_info_path):
    clinical_data = pd.read_csv(sample_info_path)
    print("Clinical data loaded from sample_info.csv")
    # Display sample characteristics to identify relevant rows
    print("Sample characteristics preview:")
    for i, row in clinical_data.iterrows():
        unique_values = clinical_data.iloc[i, 1:].unique()
        sample_chars[i] = list(unique_values)
    print(sample_chars)
else:
    print("Clinical data not available in standard files")

# Check for background information
background_path = os.path.join(in_cohort_dir, "background.txt")
if os.path.exists(background_path):
    with open(background_path, 'r') as f:
        background_info = f.read()
    print("Background information preview:")
    print(background_info[:500] + "..." if len(background_info) > 500 else background_info)
else:
    print("Background information not available")

# Set trait_row, age_row, and gender_row based on available data
# Initially set all to None (indicating data not available)
trait_row = None
age_row = None
gender_row = None

# Function to convert trait values (for allergies)
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (1 for allergic/positive, 0 for control/negative)
    if isinstance(value, str):
        value_lower = value.lower()
        if any(term in value_lower for term in ['allergic', 'allergy', 'positive', 'yes', 'disease']):
            return 1
        elif any(term in value_lower for term in ['control', 'healthy', 'negative', 'no', 'normal']):
            return 0
    
    return None

# Function to convert age values
def convert_age(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numeric age
    if isinstance(value, str):
        # Try to find numbers in the string
        numbers = re.findall(r'\d+(?:\.\d+)?', value)
        if numbers:
            try:
                return float(numbers[0])
            except:
                pass
    elif isinstance(value, (int, float)):
        return float(value)
    
    return None

# Function to convert gender values
def convert_gender(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0 for female, 1 for male)
    if isinstance(value, str):
        value_lower = value.lower()
        if any(term in value_lower for term in ['female', 'f', 'woman', 'girl']):
            return 0
        elif any(term in value_lower for term in ['male', 'm', 'man', 'boy']):
            return 1
    
    return None

# Determine if trait data is available
# This should be False if clinical_data is None or we couldn't identify a trait_row
is_trait_available = (clinical_data is not None) and (trait_row is not None)

# Save metadata for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract and save clinical features if trait data is available
if is_trait_available:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    print("Selected clinical features preview:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print(f"Clinical data processing skipped: trait data available: {is_trait_available}")


### Step 4: Gene Data Extraction

In [ ]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


### Step 5: Gene Identifier Review

In [ ]:
# Looking at the gene identifiers, I can see they're in the format of Affymetrix probe IDs (e.g., "1007_s_at")
# These are not human gene symbols but probe identifiers from an Affymetrix microarray platform
# They need to be mapped to standard gene symbols for consistent analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify the relevant columns in the gene annotation dataframe
# From previous outputs, I can see:
# - The gene identifiers in the expression data are Affymetrix probe IDs (e.g., "1007_s_at")
# - In the annotation dataframe, these are stored in the 'ID' column
# - Gene symbols are stored in the 'Gene Symbol' column

# 2. Create the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, "ID", "Gene Symbol")
print(f"Created mapping dataframe with {len(mapping_df)} probe-to-gene mappings")

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted probe-level data to gene expression data with {len(gene_data)} genes")

# Preview the first few genes in the mapped data
print("\nFirst 5 genes in the gene expression data:")
print(gene_data.index[:5])

# Check the shape of the resulting gene expression dataframe
print(f"\nShape of gene expression data: {gene_data.shape}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. First get the file paths again
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Re-extract the gene expression data
gene_data_raw = get_genetic_data(matrix_file)

# 3. Re-extract gene annotation and create mapping
gene_annotation = get_gene_annotation(soft_file)
mapping_df = get_gene_mapping(gene_annotation, "ID", "Gene Symbol")
gene_data = apply_gene_mapping(gene_data_raw, mapping_df)

# 4. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 5. Define the trait row based on the sample characteristics dictionary observed in Step 1
trait_row = 1  # The disease information is in row 1

def convert_trait(value):
    """Convert disease information to a binary indicator for allergies"""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Based on the study design, this dataset focuses on atopic dermatitis (a form of allergy)
    # and its overlap with psoriasis
    if isinstance(value, str):
        value_lower = value.lower()
        if 'atopic_dermatitis' in value_lower:
            return 1  # Allergic condition
        elif 'mixed' in value_lower:
            return 1  # This represents overlap phenotype which includes allergic component
        elif 'psoriasis' in value_lower:
            return 0  # Non-allergic skin condition
        elif 'normal_skin' in value_lower:
            return 0  # No allergic condition
    return None

# Age and gender are not available
age_row = None
gender_row = None
convert_age = None
convert_gender = None

# Reconfirm trait availability
is_trait_available = trait_row is not None
is_gene_available = True

# 6. Get the clinical data again and extract features
print("Loading the original clinical data...")
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=None,
    gender_row=gender_row,
    convert_gender=None
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 7. Link clinical and genetic data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 8. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 9. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 10. Final validation
note = "Dataset contains gene expression data from skin biopsies comparing psoriasis, atopic dermatitis, and an overlap phenotype. For allergy analysis, atopic dermatitis and the mixed phenotype are considered as cases (1), while psoriasis and normal skin are controls (0)."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 11. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")